# Importing Dependencies

In [1]:
from datetime import date, datetime
import requests
import pandas as pd
import os
import json
import re
import requests
import sqlalchemy
from sqlalchemy.orm import sessionmaker
import sqlite3

# Extract

In [2]:
# Extracting JSON data from public API of New York City website
def _extract():
    url = "https://data.cityofnewyork.us/resource/rc75-m7u3.json"
    result_load = requests.get(url)
    df = pd.DataFrame(json.loads(result_load.content))
    df.to_csv("covid_db_original_{}.csv".format(date.today().strftime("%Y%m%d")))
_extract()

# Transform

In [3]:
# df = pd.DataFrame(json.loads(result_load.content))
def _transform():
    df1 = pd.read_csv("covid_db_original_{}.csv".format(date.today().strftime("%Y%m%d")))
    df1['date'] = df1['date_of_interest'].str.extract('(....-..-..)', expand=True)
    df1.drop(df1.columns.difference(['date','case_count','hospitalized_count','death_count']), axis=1, inplace=True)
    df1 = df1.set_index("date")
    df1.to_csv("covid_db_transformed_{}.csv".format(date.today().strftime("%Y%m%d")))
_transform()

# Load

In [6]:
def _load():
    
    df2 = pd.read_csv("covid_db_transformed_{}.csv".format(date.today().strftime("%Y%m%d")))
    
    DATABASE_LOCATION = "sqlite:///covid_db_cleaned.sqlite"

    engine = sqlalchemy.create_engine(DATABASE_LOCATION)
    conn = sqlite3.connect('covid_db_cleaned.sqlite')
    cursor = conn.cursor()

    sql_query = """
    CREATE TABLE IF NOT EXISTS covid_db_cleaned (
            date DATE,
            case_count INT,
            hospitalized_count INT,
            death_count INT,
            PRIMARY KEY (date)
    )
    """

    cursor.execute(sql_query)
    print("Opened database successfully")

    try:
        df2.to_sql("covid_db_cleaned", engine, index=False, if_exists='append',con=conn)
    except:
        print("Data already exists in the database")

    conn.close()
    print("Close database successfully")
_load()

Opened database successfully
Data already exists in the database
Close database successfully
